In [27]:
# Connect to MongoDB
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

with open('../data_base_key.txt', 'r') as file:
    file_content = file.read()

uri = file_content

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


## Read Data

In [28]:
db = client['db_products']
collection = db['products']

In [38]:
for document in collection.find():
    print(document)

{'_id': ObjectId('677b1fbe32cf50de96c14463'), 'latitude': -22.19, 'longitude': -48.79, 'number of installments': 1, 'payment type': 'cartao_credito', 'place of purchase': 'SP', 'price': 43.84, 'product': 'Iniciando em programação', 'product category': 'livros', 'purchase evaluation': 5, 'seller': 'Mariana Ferreira', 'shipping fee': 0.0, 'purchase data': '01/01/2020'}
{'_id': ObjectId('677b1fbe32cf50de96c1446e'), 'latitude': -22.19, 'longitude': -48.79, 'number of installments': 1, 'payment type': 'cupom', 'place of purchase': 'SP', 'price': 1100.42, 'product': 'Tablet ABXY', 'product category': 'eletronicos', 'purchase evaluation': 1, 'seller': 'Beatriz Moraes', 'shipping fee': 68.4513476183, 'purchase data': '01/01/2021'}
{'_id': ObjectId('677b1fbe32cf50de96c14474'), 'latitude': -22.19, 'longitude': -48.79, 'number of installments': 1, 'payment type': 'boleto', 'place of purchase': 'SP', 'price': 36.84, 'product': 'Blocos de montar', 'product category': 'brinquedos', 'purchase evaluat

In [30]:
collection.update_many({}, {'$rename': {'Produto': 'product',
                                        'Categoria do Produto': 'product category',
                                        'Preço': 'price',
                                        'Frete': 'shipping fee',
                                        'Data da Compra': 'purchase data',
                                        'Vendedor': 'seller',
                                        'Local da compra': 'place of purchase',
                                        'Avaliação da compra': 'purchase evaluation',
                                        'Tipo de pagamento': 'payment type',
                                        'Quantidade de parcelas': 'number of installments',
                                        'lat': 'latitude',
                                        'lon': 'longitude'}})

In [31]:
collection.find_one()

{'_id': ObjectId('677b1fbe32cf50de96c14463'),
 'latitude': -22.19,
 'longitude': -48.79,
 'number of installments': 1,
 'payment type': 'cartao_credito',
 'place of purchase': 'SP',
 'price': 43.84,
 'product': 'Iniciando em programação',
 'product category': 'livros',
 'puchase data': '01/01/2020',
 'purchase evaluation': 5,
 'seller': 'Mariana Ferreira',
 'shipping fee': 0.0}

In [32]:
collection.distinct('product category')

['brinquedos',
 'eletrodomesticos',
 'eletronicos',
 'esporte e lazer',
 'instrumentos musicais',
 'livros',
 'moveis',
 'utilidades domesticas']

In [39]:
query = {'product category': 'livros'}

book_list = []

for document in collection.find(query):
    book_list.append(document)

In [40]:
import pandas as pd

books_df = pd.DataFrame(book_list)

In [41]:
collection.update_many({}, {'$rename': {'puchase data': 'purchase data'}})

In [42]:
books_df['purchase data'] = pd.to_datetime(books_df['purchase data'], format='%d/%m/%Y')
books_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742 entries, 0 to 741
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   _id                     742 non-null    object        
 1   latitude                742 non-null    float64       
 2   longitude               742 non-null    float64       
 3   number of installments  742 non-null    int64         
 4   payment type            742 non-null    object        
 5   place of purchase       742 non-null    object        
 6   price                   742 non-null    float64       
 7   product                 742 non-null    object        
 8   product category        742 non-null    object        
 9   purchase evaluation     742 non-null    int64         
 10  seller                  742 non-null    object        
 11  shipping fee            742 non-null    float64       
 12  purchase data           742 non-null    datetime64

In [43]:
books_df['purchase data'] = books_df['purchase data'].dt.strftime('%Y-%m-%d')

In [44]:
books_df.to_csv('../data-processed/book_table.csv', index=False)

In [45]:
query = {'purchase data': {'$regex': '/202[1-9]'}}

products_list = []

for document in collection.find(query):
    products_list.append(document)


In [46]:
products_df = pd.DataFrame(products_list)

In [47]:
products_df['purchase data'] = pd.to_datetime(products_df['purchase data'], format='%d/%m/%Y')
products_df['purchase data'] = products_df['purchase data'].dt.strftime('%Y-%m-%d')

In [48]:
products_df.head()

,_id,latitude,longitude,number of installments,payment type,place of purchase,price,product,product category,purchase evaluation,seller,shipping fee,purchase data
0,677b1fbe32cf50de96c1446e,-22.19,-48.79,1,cupom,SP,1100.42,Tablet ABXY,eletronicos,1,Beatriz Moraes,68.451348,2021-01-01
1,677b1fbe32cf50de96c14474,-22.19,-48.79,1,boleto,SP,36.84,Blocos de montar,brinquedos,4,Pedro Gomes,0.000000,2022-01-01
2,677b1fbe32cf50de96c14476,-18.10,-44.38,3,cartao_credito,MG,607.49,Fogão,eletrodomesticos,5,Juliana Costa,33.235430,2022-01-01
3,677b1fbe32cf50de96c14478,-13.29,-41.71,1,boleto,BA,1031.24,Bateria,instrumentos musicais,5,Bruno Rodrigues,55.199204,2023-01-01
4,677b1fbe32cf50de96c1447a,-22.19,-48.79,20,cartao_credito,SP,888.09,Sofá retrátil,moveis,4,Rafael Costa,51.768159,2023-01-01


In [49]:
products_df.to_csv('../data-processed/products_sales_2021_present.csv')

In [50]:
client.close()